# Building Ships

Ships are large structures that can be built in the `industrial_complex`. They consume resources to make and must be stored in a `shipyard`. 

In [1]:
import sys, os
import pandas as pd
import altair as alt
import yaml
sys.path.append('../..')

import  ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 

nest_asyncio.apply()

PII_cols = ['username','userguid','owner']

In [2]:
from app.objects import baseobjects
from app.objects import ships
from app.objects import population

In [3]:
from app.connectors.cmdb_graph import CosmosdbClient
c = CosmosdbClient()

executing local windows deployment
something wrong with your query: <class 'Exception'>


industrial_complex `has_buttons`, which can `build_ship`. Ships are built from `designs`, which are also in the config. 

In [4]:
ship_configurations = ships.ship_configurations
ship_configurations.keys()

dict_keys(['explanation', 'components', 'designs'])

A ships effort is the sum of it's components

In [5]:
probe = ship_configurations['designs']['probe']
probe

{'type': 'probe',
 'label': 'ship',
 'name': 'Probe',
 'description': 'One way scanning machine to observe a location. Requires ',
 'components': ['engine', 'scanner']}

# The Ship object

In [6]:
class Component(baseobjects.Baseobject):
    def __init__(self, config):
        super().__init__()
        self.label = "component"
        self.config = config
        self.type = config['type']
        self.name = config['name']
    

    def get_data(self):
        fund = self.get_fundimentals()
        for k,v in self.config.items():
            fund[k] = v
        fund['type'] = self.config['type']
        return fund


In [7]:
class Design(baseobjects.Baseobject):
    def __init__(self, config):
        super().__init__()
        self.label = "design"
        self.config = config
        self.type = config['type']
        self.name = config['name']
        self.design_type = config['label']
    

    def get_data(self):
        fund = self.get_fundimentals()
        for k,v in self.config.items():
            fund[k] = v
        fund['type'] = self.config['type']
        fund['label'] = self.label
        return fund


In [8]:
class Ship(baseobjects.Baseobject):
    def __init__(self, design, component_configurations):
        super().__init__()
        self.label = "ship"
        # TODO: Naming for ships
        self.name = design['name']
        self.type = design['type']
        self.design = Design(design)
        self.components = [Component(component_configurations[c]) for c in design['components']]
        self.stats = {}
        self.build_stats()
    
    def build_stats(self):
        for i in self.components:
            c = i.get_data()
            if 'augments_ship_stats' in c.keys():
                for k,v in c['augments_ship_stats'].items():
                    if k in self.stats.keys():
                        self.stats[k] += v
                    else:
                        self.stats[k] = v

    
    def get_upload_data(self):
        data = {'nodes':[], 'edges':[]}
        data['nodes'].append(self.design.get_data())
        for i in self.components:
            data['nodes'].append(i.get_data())
            data['edges'].append({'source':self.objid, 'target':i.objid, 'label':'has'})
        data['nodes'].append(self.get_data())
        return data

    def get_data(self):
        fund = self.get_fundimentals()
        for k,v in self.stats.items():
            fund[k] = v
        fund['type'] = self.type
        return fund



In [9]:
ship = Ship(probe, ship_configurations['components'])
ship

<ship: probe; 1972430856265; Probe>

In [10]:
ship.components

[<component: engine; 8727185535932; Engine>,
 <component: scanner; 3175288726804; Scanner>]

In [11]:
ship.stats

{'speed': 1, 'build_effort': 2, 'cost': 2}

In [12]:
ship.get_data()

{'name': 'Probe',
 'objid': '1972430856265',
 'label': 'ship',
 'speed': 1,
 'build_effort': 2,
 'cost': 2,
 'type': 'probe'}

In [13]:
pd.DataFrame(ship.get_upload_data()['nodes'])

,name,objid,label,type,description,components,augments_ship_stats,fuel,render,grants_capability,speed,build_effort,cost
0,Probe,0034532612170,design,probe,One way scanning machine to observe a location...,"[engine, scanner]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Engine,8727185535932,component,engine,Engines for the ship. Increases the speed and ...,NaN,"{'speed': 1, 'build_effort': 1, 'cost': 1}",0.0,cone,NaN,NaN,NaN,NaN
2,Scanner,3175288726804,component,scanner,Scans the environment for resources.,NaN,"{'cost': 1, 'build_effort': 1}",NaN,ball,[scan],NaN,NaN,NaN
3,Probe,1972430856265,ship,probe,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,2.0


In [14]:
pd.DataFrame(ship.get_upload_data()['edges'])

,source,target,label
0,1972430856265,8727185535932,has
1,1972430856265,3175288726804,has


## The Construction process:
1. user will click on the `build_ship` button in the UX. 
2. The server will validate that the faction can build a ship. 
2. ajax will take the building info and create a task.
3. a job is created for the build
4. upon completion, the ship is added to whichever `shipyard` exists in that faction

* The ajax button is here: app\templates\app\ajax\building_action.js
* The Backend Function is here: app\models\buildings.py

In [26]:
message = {'agent': {'objid': '9662163071808', 'type': 'manufacturing', 'name': 'Industrial manufacturing complex', 'faction_augments': '{wealth: -1}', 'description': 'Builds ships and other large objects for the faction.', 'has_buttons': '[build_ship]', 'current_design': 'probe', 'id': '9662163071808', 'owner': '5657120867640', 'objtype': '<->'}, 'action': 'build_ship'}
message

{'agent': {'objid': '9662163071808',
  'type': 'manufacturing',
  'name': 'Industrial manufacturing complex',
  'faction_augments': '{wealth: -1}',
  'description': 'Builds ships and other large objects for the faction.',
  'has_buttons': '[build_ship]',
  'current_design': 'probe',
  'id': '9662163071808',
  'owner': '5657120867640',
  'objtype': '<->'},
 'action': 'build_ship'}

In [16]:
agent = message['agent']
if agent['current_design'] == 'probe':
    design_config = ship_configurations['designs']['probe']

In [17]:
design_config

{'type': 'probe',
 'label': 'ship',
 'name': 'Probe',
 'description': 'One way scanning machine to observe a location. Requires ',
 'components': ['engine', 'scanner']}

In [18]:
design_config['components']

['engine', 'scanner']

In [19]:
ship = Ship(design_config, ship_configurations['components'])
ship

<ship: probe; 0479737364650; Probe>

In [20]:
ship.stats

{'speed': 1, 'build_effort': 2, 'cost': 2}

In [21]:
agent

{'objid': '9662163071808',
 'type': 'manufacturing',
 'name': 'Industrial manufacturing complex',
 'faction_augments': '{wealth: -1}',
 'description': 'Builds ships and other large objects for the faction.',
 'has_buttons': '[build_ship]',
 'current_design': 'probe',
 'id': '9662163071808',
 'owner': '5657120867640',
 'objtype': '<->'}

In [22]:
faction_buildings_query = f"g.V().has('objid','{agent['owner']}').out('isIn').in('isIn').out('owns').values('type')"
c.run_query(faction_buildings_query)    
print("faction has shipyard: ", ('shipyard' in c.res))
c.res


faction has shipyard:  False


['farmland', 'forestry', 'manufacturing']

In [25]:
from app.ajaxviews import pop

pop.build_ship(c,message)

NameError: name 'agent' is not defined